In [1]:
import pandas as pd

**Define the path for csv files. here, they are in the current working folder

In [2]:
http_req_duration_path = 'metric_http_req_duration.csv'
http_req_failed_path = 'metric_http_req_failed.csv'
vus_path = 'metric_vus.csv'

**Read csv filse as pandas data frames and keep only necessary columns

In [3]:
http_req_duration = pd.read_csv(http_req_duration_path, usecols=['time','scenario', 'count', 'mean'], dtype={'count': 'int32', 'mean': 'float32'})
http_req_failed = pd.read_csv(http_req_failed_path, usecols=['time', 'scenario','count', 'nz_count'], dtype={'count': 'int32', 'nz_count': 'int32'})
vus = pd.read_csv(vus_path, usecols=['time', 'mean'], dtype={'mean': 'float32'})

**rename the columns

In [4]:
http_req_duration = http_req_duration.rename(columns={'mean': 'response_time', 'count': 'number_of_requests'})
http_req_failed = http_req_failed.rename(columns={'count': 'failed_request_count', 'nz_count': 'failed_request_nz_count'})
vus = vus.rename(columns={'mean': 'vus_mean'})

**Time column includes the date as well. here, I've converted the time column to datetime and kept only the time value

In [5]:
http_req_duration['time'] = pd.to_datetime(http_req_duration['time']).dt.time
http_req_failed['time'] = pd.to_datetime(http_req_failed['time']).dt.time
vus['time'] = pd.to_datetime(vus['time']).dt.time

**Create a new column for Failure Rate

In [6]:
http_req_failed['failure_rate'] = http_req_failed['failed_request_nz_count'] / http_req_failed['failed_request_count']

**there are many duplicate values in "http_req_duration" datafarame.for an example, when time is 15:39:51 and scenario is "root" there are severl response time values.to avoid generating duplicates during the merging, I grouped them by based on both time and scenario columns.the same method was used  for the "http_req_failed" dataframe. 

In [7]:
http_req_duration = http_req_duration.groupby(['time','scenario']).agg({
    'number_of_requests': 'sum',
    'response_time': 'mean'
}).reset_index()


In [8]:
http_req_failed = http_req_failed.groupby(['time','scenario']).agg({
    'failed_request_count': 'sum',
    'failed_request_nz_count': 'sum',
    'failure_rate': 'mean'
}).reset_index()


**here http_req_duration and http_req_failed merged based on both time and scenario columns

In [9]:
final_metrics2_req_data = pd.merge(
    http_req_duration,
    http_req_failed,
    on=['time', 'scenario'],  
    how='left'
)

**here, converted the merged data set in to CSV file

In [10]:
final_metrics2_req_data.to_csv('final_metrics2_req_data.csv', index=False)

**then, again merged it with "VUS" dataframe

In [11]:
metrics_data = pd.merge(final_metrics2_req_data, vus, on='time', how='left')

**here, I've converted the final merged data set in to a CSV file for use in Power BI

In [12]:
metrics_data.to_csv('metrics_data.csv', index=False)

In [ ]:
#pip install mysql-connector-python

In [13]:
import mysql.connector

**set the Mysql Connection

In [14]:
connection = mysql.connector.connect(
    host="localhost",
    user="user name", 
    password="password",
    database="metrics"
)

In [15]:
cursor = connection.cursor()

**Drop the "merged_metrics" Table if its already exists to avoid duplicate table

In [16]:
cursor.execute("DROP TABLE IF EXISTS merged_metrics")

**Create the table in Mysql

In [17]:
create_table_in_Mysql = """
CREATE TABLE IF NOT EXISTS merged_metrics (
    id INT AUTO_INCREMENT PRIMARY KEY,
    time TIME,
    scenario VARCHAR(100),
    response_time FLOAT,
    failed_request_count INT,
    failed_request_nz_count INT,
    failure_rate FLOAT,
    number_of_requests INT,
    vus_mean INT
)
"""

In [19]:
cursor.execute(create_table_in_Mysql)

**Insert data in to the Table

In [20]:
for index, row in metrics_data.iterrows():
    insert_query = """
    INSERT INTO merged_metrics (time, scenario, response_time, failed_request_count, failed_request_nz_count, failure_rate, number_of_requests, vus_mean)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(insert_query, (
        row['time'],
        row['scenario'],
        row['response_time'],
        row['failed_request_count'],
        row['failed_request_nz_count'],
        row['failure_rate'],
        row['number_of_requests'],
        row['vus_mean']
    ))

connection.commit()

In [21]:
cursor.close()
connection.close()